In [ ]:
# Download a sample image to test the inference
!wget -O cat.jpg https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg
# DOwnload imagenet labels json
!wget - O imagenet_labels.json https: // raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json

In [1]:
# Load the labels
import json
with open('imagenet_labels.json') as f:
    labels = json.load(f)

In [2]:
import torch
from torchvision import models
import time

In [20]:
# Getting the model
model = models.googlenet(pretrained=True)

/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /home/venom/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [21]:
# Load the image, resize it and convert it to a tensor and display it
from PIL import Image
from torchvision import transforms

img = Image.open('cat.jpg')
img = transforms.Resize((224, 224))(img)
img = transforms.ToTensor()(img)
# The image should be normalized according to the pytorch docs
img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)

## CPU

In [22]:
model.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [23]:
# Running Inference once

start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Inference time: ', end - start)

Inference time:  0.05190610885620117


In [24]:
# Running inference 100 times
start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')

Time: 2.5829074382781982 seconds


## GPU

In [25]:
# Same in GPU
model = model.cuda()
img = img.cuda()

In [26]:
# Single image inference
start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Inference time: ', end - start)

Inference time:  0.03052520751953125


In [27]:

start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')


Time: 0.6287662982940674 seconds


## Inference Results

In [28]:
# Getting the top 5 predictions
_, indices = torch.sort(out, descending=True)
[(labels[idx], out[0, idx].item()) for idx in indices[0][:5]]


[('Egyptian Mau', 8.084476470947266),
 ('tiger cat', 7.620514869689941),
 ('tabby cat', 7.053215026855469),
 ('lynx', 6.031893730163574),
 ('plastic bag', 4.472537994384766)]

## Conclusion

Its a known fact that GPUs make AI inference faster. In this case, it gave about 3 - 4x speedup. But we can go more!

See you in the next notebook :)
